&emsp;&emsp;BPTT 算法将循环神经网络看作一个展开的多层前馈网络,其中“每一层”对应循环网络中的"每个时刻".这样，循环神经网络就可以按照前馈网络中的反向传播算法计算参数梯度.在"展开"的前馈网络中,所有层的参数是共享的,因此参数的真实梯度是所有"展开层""的参数梯度之和.

&emsp;&emsp;给定一个训练训练样本$(\mathbf{x},y)$,其中$X_{1: T}=\left(\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{t}, \ldots, \mathbf{x}_{T}\right)$为长度是$T$的输入序列,易知循环网络在时刻$t$的更新公式为

\begin{align}
\mathbf{z}_t &= W_{hh} \mathbf{h}_{t-1} + W_{ih} \mathbf{x}_t + \mathbf{b}_{ih} +  \mathbf{b}_{hh}  \\
\mathbf{h}_{t} &= f(\mathbf{z}_t)
\end{align}

其中$f(*)$是非线性激活函数,$\mathbf{y}_{1: M}=\left(y_{1}, y_{2}, \ldots, y_{m}, \ldots, y_{M}\right)$是长度为$T$的标签序列,即在每个时刻$t$,都有一个监督信息$y_t$,定义时刻$t$的损失函数为

$$ \mathcal{L}_t = \mathcal{L}(y_t, g(\mathbf{h}_t)) $$

其中$g(\mathbf{h}_t)$为第$t$时刻的输出,$\mathcal{L}$为可微分的损失函数,比如交叉熵.那么整个序列的损失函数为

$$ \mathcal{L} = \sum_{t=1}^T \mathcal{L}_t $$

整个训练的损失函数关于参数$W_{hh}$的梯度为

$$ \frac{\partial \mathcal{L}}{\partial W_{hh}}=\sum_{t=1}^{T} \frac{\partial \mathcal{L}_{t}}{\partial W_{hh}}  \tag{0}$$

即每个时刻损失$\mathcal{L}_t $对参数$W_{hh}$的偏导数之和.

&emsp;&emsp;先计算第$t$时刻损失函数$\mathcal{L}_t$关于每个参数$[w_{hh}]_{ij}$的偏导数为:

$$ \frac{\partial \mathcal{L}_{t}}{\partial [w_{hh}]_{ij}}=\sum_{k=1}^{t} \frac{\partial \mathbf{z}_{k}}{\partial [w_{hh}]_{ij}} \frac{\partial \mathcal{L}_{t}}{\partial \mathbf{z}_{k}} \tag{1} $$

首先

\begin{align}
\frac{\partial \mathbf{z}_{k}}{\partial [w_{hh}]_{ij}} &= \left[0, \cdots, \mathbf{h}_{k-1;j},\cdots, 0 \right]_i  \qquad \qquad \text{注:分布布局}  \\
&= \mathbb{I}_i( [\mathbf{h}_{k-1}]_j) \tag{2}
\end{align}

其中$\mathbb{I}_i( [\mathbf{h}_{k-1}]_j)$表示第$i$个元素值为$\mathbf{h}_{k-1}$的第$j$个元素,其余都为0的行向量

其次,定义误差项$\boldsymbol{\delta}_{t, k}=\frac{\partial \mathcal{L}_{t}}{\partial \mathbf{z}_{k}} $为第$t$时刻的损失对第$k$时刻隐藏神经层的净输入$\mathbf{z}_k$的导数,则当$1 \leq k <t$时,有

\begin{align}
\boldsymbol{\delta}_{t, k} &=\frac{\partial \mathcal{L}_{t}}{\partial \mathbf{z}_{k}} \\
              &=\frac{\partial \mathbf{h}_{k}}{\partial \mathbf{z}_{k}} \frac{\partial \mathbf{z}_{k+1}}{\partial \mathbf{h}_{k}} \frac{\partial \mathcal{L}_{t}}{\partial \mathbf{z}_{k+1}} \\
&=
\begin{bmatrix}
\frac{ \partial f([z_{k}]_1)}{\partial [z_{k}]_1} & 0 & \cdots & 0  \\
0 & \frac{ \partial f([z_{k}]_2)}{\partial [z_{k}]_2} & \cdots & 0 \\
\vdots& \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \frac{ \partial f([z_{k}]_{-1})}{\partial [z_{k}]_{-1}}
\end{bmatrix}  W_{hh}^{T} \boldsymbol{\delta_{\mathit{t, k+1}}} \\ \\\
&=A_k W_{hh}^{\top} \boldsymbol{\delta}_{t, k+1} \qquad \qquad \text{注:列向量}
\end{align}

将公式(2)公式(3)代入公式(1)可得

$$ \frac{\partial \mathcal{L}_{t}}{\partial [w_{hh}]_{ij}}=\sum_{k=1}^{t}\left[\delta_{t, k}\right]_{i}\left[\mathbf{h}_{k-1}\right]_j  \tag{3}$$

将上式写成矩阵形式有

$$ \frac{\partial \mathcal{L}_{t}}{\partial W_{hh}}=\sum_{k=1}^{t}\boldsymbol{\delta}_{t, k} \mathbf{h}_{k-1}^T \tag{4} $$

下图给出了误差项随时间进行反向传播算法的示例

<img src='../../../../../Other/img/BPTT.png'>

将公式(4)代入公式(0),得到整个序列的损失函数$\mathcal{L}$关于参数$W_{hh}$的梯度

$$ \frac{\partial \mathcal{L}}{\partial W_{hh}}= \sum_{t=1}^T \sum_{k=1}^{t}\boldsymbol{\delta}_{t, k} \mathbf{h}_{k-1}^T \tag{5}  $$


同理可得,$\mathcal{L}$关于权重$W_{ih}$和偏置$\mathbf{b}_{hh},\mathbf{b}_{ih}$的梯度为

\begin{align}
\frac{\partial \mathcal{L}}{\partial W_{ih}}    &= \sum_{t=1}^T  \sum_{k=1}^{t} \boldsymbol{\delta}_{t, k} \mathbf{x}_{k}^T   \tag{6} \\
\frac{\partial \mathcal{L}}{\partial \mathbf{b}} &= \sum_{t=1}^T  \sum_{k=1}^{t} \boldsymbol{\delta}_{t, k}  \tag{7} 
\end{align}